In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Evaluating CNN+RNN models on the dataset

# Imports
import gc
import os, random
from collections import Counter
from datetime import datetime

import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from wandb.keras import WandbCallback

import wandb
from wp8.pre_processing.utils import safe_mkdir
from wp8.utils.cnn_rnn_utils import load_and_split
from wp8.utils.dataset import TimeSeriesGenerator as TSG


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [3]:
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)


In [4]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""

    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


opt = dotdict(
    {
        "lstm1_units": 128,
        "lstm2_units": 32,
        "dense_units": 32,
        "dropout": 0.5,
        "epochs": 1000,
        "train_actors": [1, 2, 3],
        "val_actors": [4],
        "train_cams": [1, 2, 3],
        "val_cams": [1],
        "seq_len": 20,
        "split_ratio": None,
        "drop_offair": False,
        "undersample": False,
        "batch_size": 40,
        "stride": 10,
        "learning_rate": 1e-4,
        "micro_classes": False,
    }
)


In [5]:
X_train, y_train, X_val, y_val, cams_train, cams_val = load_and_split(
    opt.train_actors, opt.val_actors, opt.train_cams, opt.val_cams, opt.split_ratio, opt.drop_offair, opt.undersample, opt.micro_classes
)
print(f"\nX_train shape: {X_train.shape}, len y_train: {len(y_train)}, X_val shape: {X_val.shape}, len y_val: {len(y_val)}\n")


[STATUS] Load Train Set


Loading features: outputs/dataset/features/Actor_1_Walk_PH.npz:  30%|██▉       | 13/44 [00:14<00:41,  1.33s/it]           

In [ ]:
series_gen = TSG(opt)
X_train_series, y_train_series, class_weights, classes = series_gen.get_train_series(X_train, y_train, cams_train)
X_val_series, y_val_series = series_gen.get_val_series(X_val, y_val, cams_val)
print(f"\nX_train_series shape: {X_train_series.shape}, len y_train_series: {len(y_train_series)}, X_val_series shape: {X_val_series.shape}, len y_val_series: {len(y_val_series)}\n")


In [ ]:
print(f"Before resampling - y_train_series shape: {Counter(y_train_series)}")


### Resampling


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
from imblearn.combine import SMOTETomek
import copy

undersampling_strategy = {0: 6000, 2: 6000, 1: 6000}
oversampling_strategy = {0: 23890, 2: 10000, 1: 10000}
under = RandomUnderSampler(random_state=2, sampling_strategy=undersampling_strategy)  
over = RandomOverSampler(random_state=2, sampling_strategy=oversampling_strategy)  
steps = [("o", over), ("u", under)]
pipeline = Pipeline(steps=steps)
# smt = SMOTETomek(random_state=2)


In [ ]:
print(f"X_train_series.shape: {X_train_series.shape}")

np.reshape(X_train_series, (-1, 20 * 2048))
print(f"X_train_series.shape: {X_train_series.shape}")

X_train_series, y_train_series = pipeline.fit_resample(X_train_series, y_train_series)

np.reshape(X_train_series, (-1, 20, 2048))

print(f"After resampling X_train_series.shape: {X_train_series.shape}")
print(f"After resampling - y_train_series count: {Counter(y_train_series)}")


In [ ]:
# WANDB project initialization
run = wandb.init(
    project="Fall detection CNN + RNN",
    config={
        "model": "LSTM",
        "epochs": opt.epochs,
        "seq_len": opt.seq_len,
        "num_features": 2048,
        "batch_size": opt.batch_size,
        "stride": opt.stride,
        "loss_function": "sparse_categorical_crossentropy",
        "architecture": "LSTM",
        "train_actors": opt.train_actors,
        "val_actors": opt.val_actors,
        "train_cams": opt.train_cams,
        "val_cams": opt.val_cams,
        "micro_classes": opt.classes,
        "dropout": opt.dropout,
        "lstm1_units": opt.lstm1_units,
        "lstm2_units": opt.lstm2_units,
        "dense_units": opt.dense_units,
        "learning_rate": opt.learning_rate,
        "split_ratio": opt.split_ratio,
        "drop_offair": opt.drop_offair,
        "undersample": opt.undersample,
    },
)

cfg = wandb.config


In [ ]:
model = Sequential()
model.add(LSTM(units=cfg.lstm1_units, input_shape=(cfg.seq_len, cfg.num_features)))
model.add(Dropout(cfg.dropout))
# model.add(LSTM(units=cfg.lstm2_units, input_shape=(cfg.seq_len, cfg.num_features)))
# model.add(Dropout(cfg.dropout))
# model.add(Dense(units=cfg.dense_units, activation="relu"))
# model.add(Dropout(cfg.dropout))
model.add(Dense(units=np.unique(y_train_series, axis=0).shape[0], activation="softmax"))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=cfg.learning_rate),
    loss=cfg.loss_function,
    metrics=["accuracy"],
)
model.summary()


In [ ]:
# Callbacks
dir_path = f"model_checkpoints/{cfg.model}"
safe_mkdir(dir_path)
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y_%H:%M:%S")

model_checkpoint = ModelCheckpoint(
    filepath=f"{dir_path}/{cfg.model}_{dt_string}",
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
    initial_value_threshold=0.8,
    verbose=1,
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=30,
    verbose=1,
    mode="min",
    min_delta=1e-5,
    cooldown=1,
    min_lr=1e-6,
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=50,
    verbose=1,
    mode="min",
)

callbacks = [WandbCallback(), reduce_lr, early_stop]


In [ ]:
# Train Model
history = model.fit(X_train_series, y_train_series, validation_data=(X_val_series, y_val_series), epochs=cfg.epochs, callbacks=callbacks, batch_size=cfg.batch_size)


In [ ]:
# Evaluate Model
val_logits = model.predict(X_val_series, batch_size=60, verbose=1)


In [ ]:
# free up memory
# del X_train_series
# del y_train_series
# del X_val_series
# del y_val_series

# gc.collect()


In [ ]:
# Log metrics to wandb
y_pred_val_classes = np.argmax(val_logits, axis=1).tolist()

cr = classification_report(y_val_series, y_pred_val_classes, target_names=classes)

wandb.log(
    {
        "Classification Report": cr,
    }
)

wandb.sklearn.plot_roc(y_val_series, val_logits, classes)
wandb.sklearn.plot_class_proportions(y_train_series, y_val_series, classes)
wandb.sklearn.plot_precision_recall(y_val_series, val_logits, classes)
wandb.sklearn.plot_confusion_matrix(y_val_series, y_pred_val_classes, classes)
wandb.join()


In [ ]:
print(cr)
